In [10]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
import spacy
from models.SIM import SIM
import utils
import pickle
from sympy.parsing.sympy_parser import parse_expr
from sympy.parsing.sympy_parser import standard_transformations, convert_xor,implicit_multiplication

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def loadObject(file_path):
    with open(fie_path,'rb') as f:
        return pickle.load(f)

In [28]:
math_train = pd.read_json(r"..\Data\dolphin-number_word_std\number_word_std.dev.json")
math_test = pd.read_json(r"..\Data\dolphin-number_word_std\number_word_std.test.json")
len(math_train),len(math_test)

(374, 1504)

In [32]:
130/1504

0.08643617021276596

In [7]:
model = SIM()
model.fit(math_train)

SIM()

In [8]:
model.score(math_train)

0.17379679144385027